<center><h1>Python List Operations - Extend or Append?</h1></center>
<center><h3>Written 17/11/2020, by Tyler J. Russell</h3></center>

***

#### Abstract

I was in computer science class, attending a lecture involving Python list operations, when a classmate posed a very interesting question.  
To quote directly, "*Wouldn't extend be a better choice instead of append, since extend allows you to insert multiple variables into a list while append only allows for one? Or are there cases where append would be more suitable?*"

I will admit to having never heard of nor needed to use the `#list.extend()` function before this situation. Hence, it is of personal benefit to me, too, to resolve this question, as it may provide insight into an uncommon feature of Python and its usages.  
As such, this document will discuss and review that question systematically.

It is worth noting that in this instance superiority is weighed on a metric of performance, in both operational expense and time to execute.  
To ensure fairness and equality of testing, each list will end with the same number of elements, so the number of operations will equate to the base number (I am using 1 million as the base number for this series of tests) divided by the number of elements we are concatenating.

#### Hypothesis

Logically, it seems likely that `#list.extend()` will only be more efficient in situations where more than one element is present. That will be my first hypothesis: for all singular concatenation operations, `#list.append()` will be superior by our metrics.  
A derivative second hypothesis arises from this: For all concatenation operations involving more than one element, `#list.extend()` will be superior by our metrics.  
And, finally, my last hypothesis is that traditional list concatenation, as in `list1 + list2` syntax, should result to be superior to `#list.extend()` by our metrics.

#### Testing

Setting up our test suite:

In [45]:
from timeit import timeit
setup = "test_list = []"

def test(tests, setup):
    for (key, value) in tests.items():
        time = timeit(value[0], setup, number=value[1])
        print(f"{key}: {time}")

Our first hypothesis:

In [46]:
tests = {}
tests["append"] = ["test_list.append(0)", 1000000]
tests["extend"] = ["test_list.extend([0])", 1000000]

test(tests, setup)

append: 0.08677059999990888
extend: 0.1090719999999692


As shown above, our first hypothesis works out to be true; `#list.append()` is indeed faster than `#list.extend()` for operations where only one item is concatenated, by an average of ~30ms per 1 million operations.  
But now, what about our second hypothesis, where multiple elements must be concatenated:

In [47]:
tests = {}
tests["append"] = ["test_list.append(0)", 1000000]
tests["append_on_range"] = ["test_list.append(0);test_list.append(0);test_list.append(0);test_list.append(0);test_list.append(0)", 200000] # just in case, for fairness purposes
tests["extend"] = ["test_list.extend([0, 0, 0, 0, 0])", 200000]

test(tests, setup)

append: 0.08749360000001616
append_on_range: 0.08182080000005953
extend: 0.05304019999994125


Our second hypothesis also evaluates to be true; `#list.extend()` is faster for operations where multiple items are concatenated, in a fashion one would assume to be inversely proportional to the number of values added in one operation.  
Just for comparison purposes, I added in another test, which demonstrates it's actually faster, albeit very minutely, to run multiple `#list.append()` statements together less times as opposed to one call on `#list.append()` many more times.  
Finally, on to our last hypothesis:

In [55]:
tests = {}
setup = "list_one = [1]; list_two = [2]"
tests["append_single"] = ["list_one.append(list_two[0])", 1000000]
tests["extend_single"] = ["list_one.extend(list_two)", 1000000]
"""tests["concat_single"] = ["list_one = list_one + list_two", 1000000]"""

test(tests, setup)

tests = {}
setup = "list_one = [1, 2, 3, 4, 5]; list_two = [6, 7, 8, 9, 10]"
tests["append_multi"] = ["[list_one.append(x) for x in list_two]", 1000000]
tests["extend_multi"] = ["list_one.extend(list_two)", 1000000]
"""tests["concat_multi"] = ["list_one = list_one + list_two", 1000000]"""

test(tests, setup)

append_single: 0.09268669999983103
extend_single: 0.08435000000008586
append_multi: 0.6194018000001051
extend_multi: 0.23886309999988953


Our third hypothesis, on the other hand, ends up being false; the `list1 + list2` syntax, when used to add to lists in-place, gets very expensive proportional to the size of the lists being concatenated. For this reason I actually had to comment out the concatenation tests to prevent them from halting the rendering of the notebook, however if you wish to verify my claims, I have left them in the code so you can run the test suite yourself on any Python-3 supportive IPy, Jupyter or other kernel.  
It is, however, worth noting that for smaller lists, or without concatenating in-place on very large lists, you can also achieve a test result where concatenation syntax is faster.

#### Conclusion

In summary:
- `#list.append()` is faster than `#list.extend()` for single-element operations
- `#list.extend()` is faster than `#list.append()` inversely proportional to the number of elements for all others
- `list1 + list2` syntax is faster than both for small lists, but gets very performance expensive when working with large lists

Based on this information, I conclude that `#list.extend()` should be preferred in instances where multiple elements are to be added at the same time, and `#list.append()` should be preferred in instances where the number of elements is unknown or they are to be added iteratively instead of simultaneously.  

May this document serve useful to you, its reader, as this research did to me.